Import tools to help communicate with Fred API
Requests helps us send messages over the internet
Optional lets us say that sometimes a piece of information might not be there.

In [31]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.exceptions import InvalidSignature
from base64 import b64encode, b64decode

class FredPyAPI:
    def __init__(self, token: Optional[str] = None, token_signature: Optional[str] = None):
        self.token = token
        self.token_signature = token_signature
        self.public_key = None  # This should be set with a method

    def set_public_key(self, public_key_pem: str):
        from cryptography.hazmat.primitives.serialization import load_pem_public_key
        self.public_key = load_pem_public_key(public_key_pem.encode())

    def set_token(self, token: str, token_signature: str) -> None:
        if self.verify_token(token, token_signature):
            self.token = token
            self.token_signature = token_signature
        else:
            raise ValueError("Token verification failed")

    def verify_token(self, token: str, signature: str) -> bool:
        if not self.public_key:
            raise ValueError("Public key not set. Use set_public_key method first.")
        
        try:
            self.public_key.verify(
                b64decode(signature),
                token.encode(),
                padding.PSS(
                    mgf=padding.MGF1(hashes.SHA256()),
                    salt_length=padding.PSS.MAX_LENGTH
                ),
                hashes.SHA256()
            )
            return True
        except InvalidSignature:
            return False

    # ... rest of the class implementation ...

In [ ]:
# Usage example:
# fred_api = FredPyAPI()
# fred_api.set_token("your_api_key_here")
# series_data = fred_api.get_series_data("GDP")
# historical_data = fred_api.get_historical_data("GDP", observation_start="2020-01-01", observation_end="2023-12-31")
# categories = fred_api.get_categories()
# releases = fred_api.get_releases(realtime_start="2023-01-01")